# Big Data Analytics with MovieLens Dataset

In this Jupyter Notebook, we will use the [MovieLens 20M Dataset](https://grouplens.org/datasets/movielens/20m/) on movie ratings to answer several tasks by using `pySpark`. The exercises are structured as a guideline to get familiar with the Pyspark syntax. Have also a look on the [official pySpark documentation](https://spark.apache.org/docs/latest/api/python/pyspark.html). 

**Introduction to Movielens dataset**

The Introduction exercises have the following goals:
- Reading and understanding the schema of our movielens dataset
- Calculating some summary statistics of our dataset
- Learn how to perform joins and aggregations using Spark

This will be also illustrated by guided exercises to get a first understanding of Spark
- Guided Exercise 1: Which movies are the most popular ones?
- Guided Exercise 2: What are the distinct genres in the Movielens Dataset (RDD)?


**Exercises for you:**
- Exercise 1: Which movies have the highest number of ratings?
- Exercise 2: What's the number of movies in each genre?
- Exercise 3: Which movies are a matter of taste?


## Initialize Sparksession

Execute the following cell to initialize a Sparksession:

In [1]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('MovieLens').getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/29 21:16:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Introduction: Reading the dataset

Our movielens dataset contains 20 million ratings and 465'000 tag applications applied to 27'000 movies by 138'000 users. It also includs tag genome data with 12 million relevance scores across 1100 tags.

The whole dataset contains six CSV files:
- genome-scores.csv
- genome-tags.csv
- links.csv
- movies.csv
- ratings.csv
- tags.csv

In this Introduction exercise, we will have a look on the **`movies`** and **`ratings`** dataframes.

To read a CSV file in our "ml-20m" folder, we access the `DataFrameReader` class through `read` and call the `csv()` method on it. We also specify `option("header", "true")` since the first row of the file contains our column names. 

In [2]:
ratings = spark.read.option("header", "true").csv("hdfs://rpi0:8020/data/ml-20m/ratings.csv")
ratings.show(5)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|      2|   3.5|1112486027|
|     1|     29|   3.5|1112484676|
|     1|     32|   3.5|1112484819|
|     1|     47|   3.5|1112484727|
|     1|     50|   3.5|1112484580|
+------+-------+------+----------+
only showing top 5 rows



Each tuple of the `ratings` DataFrame represents one rating (`rating`) for one movie (`movieId`) by one user (`userId`). The ratings ranges from 0.5 stars (worst) up to 5.0 stars (best). 

We can also have look on the Schema of our dataset (column names and types) by using the `printSchema()` method.

In [3]:
ratings.printSchema()

root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- timestamp: string (nullable = true)



Do the same for the `movies.csv` file. What kind of data is available and how does the schema look like?

In [4]:
movies = spark.read.option("header", "true").csv("hdfs://rpi0:8020/data/ml-20m/movies.csv")
movies.show(5, truncate = False)

movies.printSchema()

+-------+----------------------------------+-------------------------------------------+
|movieId|title                             |genres                                     |
+-------+----------------------------------+-------------------------------------------+
|1      |Toy Story (1995)                  |Adventure|Animation|Children|Comedy|Fantasy|
|2      |Jumanji (1995)                    |Adventure|Children|Fantasy                 |
|3      |Grumpier Old Men (1995)           |Comedy|Romance                             |
|4      |Waiting to Exhale (1995)          |Comedy|Drama|Romance                       |
|5      |Father of the Bride Part II (1995)|Comedy                                     |
+-------+----------------------------------+-------------------------------------------+
only showing top 5 rows

root
 |-- movieId: string (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



## Guided Exercise 1: Which movies are the most popular ones?

To get the most popular movies, we are looking for the movies with the highest number of ratings. In this task, we assume the number of ratings as a representative for the most popular movies. To do this, we will perform the following *transformations* on the `ratings` DataFrame: 
- Group by `movieId`
- Count the number of users (`userId`) associated with each movie 
- Rename this column to `num_ratings`
- Sort by `num_ratings` in descending order 

We do these transformations in `pySpark` and store the DataFrame as `most_popular`. Have also a look on the [official pySpark documentation](https://spark.apache.org/docs/latest/api/python/pyspark.html).

**HINT**:
- Use `agg(count())` to perform an aggregate calculation on grouped data. 
- Don't forget that transformations are [lazy](https://spark.apache.org/docs/latest/rdd-programming-guide.html#rdd-operations) in spark. We need to call an action (e.g. `show()` for Dataframes, `take()` for RDD's) explicitly to see the results. 

In [5]:
from pyspark.sql.functions import *

most_popular = ratings.groupBy("movieId").agg(count("userId")).withColumnRenamed("count(userId)", "num_ratings").sort(desc("num_ratings"))

most_popular.show(5)

+-------+-----------+
|movieId|num_ratings|
+-------+-----------+
|    296|      67310|
|    356|      66172|
|    318|      63366|
|    593|      63299|
|    480|      59715|
+-------+-----------+
only showing top 5 rows



Unfortunately, the resulting table only contains `movieId` and `num_ratings`. The title of the movie is stored in the `movies` DataFrame. So, we need an inner join of our `most_popular` DataFrame with the `movies` DataFrame on `movieId`.

In [6]:
most_popular_movies = most_popular.join(movies, most_popular.movieId == movies.movieId)
most_popular_movies.select("title", "num_ratings").show(5, truncate = False)

+-----------------------------------------+-----------+
|title                                    |num_ratings|
+-----------------------------------------+-----------+
|Platoon (1986)                           |15808      |
|Pulp Fiction (1994)                      |67310      |
|Popeye (1980)                            |3539       |
|Last King of Scotland, The (2006)        |4163       |
|High School Musical 3: Senior Year (2008)|272        |
+-----------------------------------------+-----------+
only showing top 5 rows



We now have a list of the most popular (or most rated) movies of our movielens dataset. Have you already watched all of them? :)

## Guided Exercise 2: Understanding RDD (Resilient Distributed Datasets) operations

We will have a look on two core data abstractions of Spark, namely Dataframes and RDD's.
RDDs and DataFrames can be created from external data sources (e.g. HDFS, SQL) or from internal process steps. Dataframes the are easiest abstraction. One can compare Dataframes with a traditional table with columns and rows, which is generally used for handling workflows with structured data. If the data is unstructured (has no schema) and the data needs to be manipulated in non-standard ways, one should use RDD's. Even though our data is structured, we will use some operations on RDD's to understand RDD transformations. 

Have a look on the Pyspark Documentation for RDD operations [PySpark Package](https://spark.apache.org/docs/1.5.1/api/python/pyspark.html)



Execute the following cells and try to understand what map(), flatmap() and take() do

In [7]:
# Load the data into RDD
data = sc.textFile("hdfs://rpi0:8020/data/ml-20m/movies.csv")

# Split the RDD 
moviesRDD = data.map(lambda l: l.split(','))
moviesRDD.take(5)

[['movieId', 'title', 'genres'],
 ['1', 'Toy Story (1995)', 'Adventure|Animation|Children|Comedy|Fantasy'],
 ['2', 'Jumanji (1995)', 'Adventure|Children|Fantasy'],
 ['3', 'Grumpier Old Men (1995)', 'Comedy|Romance'],
 ['4', 'Waiting to Exhale (1995)', 'Comedy|Drama|Romance']]

In [8]:
# map
Reversed = moviesRDD.map(lambda m: m[::-1])
Reversed.take(5)

[['genres', 'title', 'movieId'],
 ['Adventure|Animation|Children|Comedy|Fantasy', 'Toy Story (1995)', '1'],
 ['Adventure|Children|Fantasy', 'Jumanji (1995)', '2'],
 ['Comedy|Romance', 'Grumpier Old Men (1995)', '3'],
 ['Comedy|Drama|Romance', 'Waiting to Exhale (1995)', '4']]

In [9]:
# flatMap
words = moviesRDD.flatMap(lambda m: list(m))
words.take(10)

['movieId',
 'title',
 'genres',
 '1',
 'Toy Story (1995)',
 'Adventure|Animation|Children|Comedy|Fantasy',
 '2',
 'Jumanji (1995)',
 'Adventure|Children|Fantasy',
 '3']

## Extracting Distinct Movie Genres

Now we will try to combine these operations above by trying to extract all distinct movie genres in our Movielens data. You need to:
- Read the csv file located in ("ml-20m/movies.csv")
- Split the data and select the corresponding genre column via `map()`
- `flatmap()` the data --> **Remark**: Be aware that a movie can contain several genres delimited by `('|')`
- Print the results by`take()` the `distinct()` genres


In [10]:
movies_rdd=sc.textFile("hdfs://rpi0:8020/data/ml-20m/movies.csv")

movies_split= movies_rdd.map(lambda lines:lines.split(','))
genres=movies_split.map(lambda line:(line[2]))
genre=genres.flatMap(lambda l:l.split('|'))
genres_distinct=genre.distinct()

print(genres_distinct.take(10))

['Adventure', 'Children', 'Comedy', 'Romance', 'Drama', 'Action', ' The (1995)"', 'Horror', 'Sci-Fi', 'IMAX']


## Exercise 1: Which movies have the highest ratings (in average)?

Now we want to see which movies are rated to be the best. We will use the `ratings` DataFrame and: 

- Group by `movieId` 
- Calculate the average rating for each movie and rename this column to `avg_rating`
- Sort by `avg_rating` in descending order 
- Join the resulting DataFrame with the `movies` DataFrame to get the movienames.

**NOTE** Be sure that you read the movies file in the first part

In [11]:
top_rated = ratings.groupBy("movieId").agg(avg(col("rating"))).withColumnRenamed("avg(rating)", "avg_rating").sort(desc("avg_rating"))

top_rated_movies = top_rated.join(movies, top_rated.movieId == movies.movieId)

top_rated_movies.show(5, truncate = False)

+-------+------------------+-------+------------------------+-------------------------------------------+
|movieId|avg_rating        |movieId|title                   |genres                                     |
+-------+------------------+-------+------------------------+-------------------------------------------+
|296    |4.174231169217055 |296    |Pulp Fiction (1994)     |Comedy|Crime|Drama|Thriller                |
|1090   |3.919977226720648 |1090   |Platoon (1986)          |Drama|War                                  |
|3959   |3.699372603694667 |3959   |Time Machine, The (1960)|Action|Adventure|Sci-Fi                    |
|2294   |3.303207714257601 |2294   |Antz (1998)             |Adventure|Animation|Children|Comedy|Fantasy|
|6731   |3.5571184995737424|6731   |Day of the Dead (1985)  |Horror|Sci-Fi|Thriller                     |
+-------+------------------+-------+------------------------+-------------------------------------------+
only showing top 5 rows



The resulting Dataframe is maybe not meaningful. We should also consider the number of ratings by doing an aggregation `agg()` call. 

In [12]:
top_rated = ratings.groupBy("movieId").agg(count("userId"), avg(col("rating"))).withColumnRenamed("count(userId)", "num_ratings").withColumnRenamed("avg(rating)", "avg_rating")

top_rated_movies = top_rated.join(movies, top_rated.movieId == movies.movieId).sort(desc("avg_rating"), desc("num_ratings"))

top_rated_movies.show(5, truncate = False)

+-------+-----------+----------+-------+---------------------------------------------------------+-----------+
|movieId|num_ratings|avg_rating|movieId|title                                                    |genres     |
+-------+-----------+----------+-------+---------------------------------------------------------+-----------+
|103871 |2          |5.0       |103871 |Consuming Kids: The Commercialization of Childhood (2008)|Documentary|
|108527 |2          |5.0       |108527 |Catastroika (2012)                                       |Documentary|
|117314 |1          |5.0       |117314 |Neurons to Nirvana (2013)                                |Documentary|
|120134 |1          |5.0       |120134 |Doggiewoggiez! Poochiewoochiez! (2012)                   |Comedy     |
|54326  |1          |5.0       |54326  |Sierra, La (2005)                                        |Documentary|
+-------+-----------+----------+-------+---------------------------------------------------------+-----------+
o

All of the movies with `avg_rating` of exactly 5.0 have 2 or less `num_ratings` . We must investigate the distribution of `num_ratings` to only consider movies that have a minimum number of ratings. Let's calculate some summary statistics within Spark.

In [13]:
# Calculate average, minimum, and maximum of num_ratings
top_rated_movies.select([mean('num_ratings'), min('num_ratings'), max('num_ratings')]).show(1)

+-----------------+----------------+----------------+
| avg(num_ratings)|min(num_ratings)|max(num_ratings)|
+-----------------+----------------+----------------+
|747.8411232425965|               1|           67310|
+-----------------+----------------+----------------+



In [14]:
top_rated_movies.where("num_ratings > 800").show(5)

+-------+-----------+-----------------+-------+--------------------+--------------------+
|movieId|num_ratings|       avg_rating|movieId|               title|              genres|
+-------+-----------+-----------------+-------+--------------------+--------------------+
|    318|      63366|4.446990499637029|    318|Shawshank Redempt...|         Crime|Drama|
|    858|      41355|4.364732196832306|    858|Godfather, The (1...|         Crime|Drama|
|     50|      47006|4.334372207803259|     50|Usual Suspects, T...|Crime|Mystery|Thr...|
|    527|      50054|4.310175010988133|    527|Schindler's List ...|           Drama|War|
|   1221|      27398|4.275640557704942|   1221|Godfather: Part I...|         Crime|Drama|
+-------+-----------+-----------------+-------+--------------------+--------------------+
only showing top 5 rows



## Exercise 2: What's the number of movies in each genre?

In this exercise, we want to calculate the number of movies in each genre. This exercise is similar to the guided introduction exercise 2:

- Read the csv file located in ("ml-20m/movies.csv")
- Split the data and select the corresponding genre column via `map()`
- `flatmap()` the data --> **Remark**: Be aware that a movie can contain several genres delimited by `('|')`
- Have a look on the [official pySpark documentation](https://spark.apache.org/docs/latest/api/python/pyspark.html) and check what the `reduceByKey()` function do. This function is needed to sum up the number of movies in each genre. 
- Sort the results using the `sortBy()` function
- Print the results by`take()`

In [15]:
movies_rdd=sc.textFile("hdfs://rpi0:8020/data/ml-20m/movies.csv")
movies_split=movies_rdd.map(lambda lines:lines.split(','))
flat_genre=movies_split.map(lambda line:(line[2]))
genre=flat_genre.flatMap(lambda l:l.split('|'))
genre_kv=genre.map(lambda k:(k,1))
genre_count=genre_kv.reduceByKey(lambda k,v: (k+v))

genre_sort= genre_count.sortBy(ascending=False, keyfunc = lambda x: x[1])

print(genre_sort.take(10))

[('Drama', 9952), ('Comedy', 6545), ('Romance', 3186), ('Thriller', 3131), ('Action', 2851), ('Crime', 2251), ('Documentary', 1979), ('Horror', 1949), ('Adventure', 1750), ('Sci-Fi', 1394)]


The movielens dataset contains 9952 movies, which belongs to the 'Drama' genre. Also the genres 'Comedy', 'Romance', and 'Thriller' are in high demand. 

## Exercise 3: Which movies are a matter of taste?

As you know, movies are a matter of taste. There are for sure some movies, which you would rate with a 5 whereas your friend rates the same movie with a 2. These are the movies that divide your opinon. Try to find out, which movies belong to this category.

**HINT**

- We need to consider the standard deviation of the movie ratings
- Also, try to consider only movies that have some minimum number of ratings (e.g. 700) 
- Join with the movies table to get the movie names

In [16]:
ratings_stddev = ratings.groupBy("movieId").agg(count("userId").alias("num_ratings"), 
avg(col("rating")).alias("avg_rating"),stddev(col("rating")).alias("std_rating")).where("num_ratings > 700")

ratings_stddev.show(5)

+-------+-----------+------------------+------------------+
|movieId|num_ratings|        avg_rating|        std_rating|
+-------+-----------+------------------+------------------+
|    296|      67310| 4.174231169217055|0.9760762295742447|
|   2162|       2086|2.4223394055608822|1.0929069291260938|
|    829|       1821|2.6765513454146075|1.1909661657407078|
|   1090|      15808| 3.919977226720648|0.8272067263021854|
|   2069|       1128| 3.806294326241135| 0.887868203627674|
+-------+-----------+------------------+------------------+
only showing top 5 rows



In [17]:
matterofTaste_movies = ratings_stddev.join(movies, ratings_stddev.movieId == movies.movieId)

matterofTaste_movies.sort(desc("std_rating")).select("title", "num_ratings", "std_rating").show(5, truncate = False)

+-----------------------------------+-----------+------------------+
|title                              |num_ratings|std_rating        |
+-----------------------------------+-----------+------------------+
|Plan 9 from Outer Space (1959)     |2304       |1.4201711823223824|
|Freddy Got Fingered (2001)         |1550       |1.3504497277537106|
|Twilight Saga: Eclipse, The (2010) |884        |1.3485057430514158|
|Twilight Saga: New Moon, The (2009)|1135       |1.3367548401080391|
|Passion of the Christ, The (2004)  |3130       |1.3354273707057587|
+-----------------------------------+-----------+------------------+
only showing top 5 rows



Without any surprise, Twilight is a highly debated movie ;) Plan 9 from Outer Space has the largest standard deviation and is thus the movie, that divides the opinion the most.  